# 1.0 Preprocessing
---

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.callbacks import Callback
import pandas as pd
import tensorflow as tf

In [2]:
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'],axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# application_types_to_replace = ['T14','T25','T15','T29','T17']
typeDF = application_df['APPLICATION_TYPE'].value_counts().to_frame().reset_index()
type10 = typeDF.loc[typeDF['APPLICATION_TYPE']<10]
application_types_to_replace = type10['index'].to_list()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1236        1
C2380        1
C2561        1
C2600        1
C6100        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C1720        6
C4100        6
C2400        6
C1600        5
C1257        5
C2710        3
C1260        3
C0           3
C1246        2
C1234        2
C1256        2
C3200        2
C1267        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classDF = application_df['CLASSIFICATION'].value_counts().to_frame().reset_index()
class10 = classDF.loc[classDF['CLASSIFICATION']<10]
classifications_to_replace = class10['index'].to_list()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Other       98
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T3,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [11]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL'],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# 2.0 Compile, Train and Evaluate the Model
---

## 2.1 Detemine Layers and Activation

### 2.1.1 - 2 Layers and Activation All Relu

In [56]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  40
hidden_nodes_layer2 = 20
# hidden_nodes_layer3 = 10
# hidden_nodes_layer4 = 5


nn1_1 = tf.keras.models.Sequential()

# First hidden layer
nn1_1.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn1_1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
# nn1_1.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
# nn1_11.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="sigmoid"))

# Output layer
nn1_1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1_1.summary()


# Compile the model
nn1_1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
mc = tf.keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', 
                                     save_freq='epoch',period=5)
fit_model = nn1_1.fit(X_train_scaled, y_train, callbacks=[mc], epochs=50)

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_71 (Dense)             (None, 40)                2880      
_________________________________________________________________
dense_72 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_73 (Dense)             (None, 1)                 21        
Total params: 3,721
Trainable params: 3,721
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 1s 508us/step - loss: 0.5817 - accuracy: 0.7145
Epoch 2/50
804/804 [==============================] - 0s 523us/step - loss: 0.5533 - accuracy: 0.7299
Epoch 3/50
804/804 [==============================] - 0s 548us/step - loss: 0.5500 - accuracy: 0.7309
Epoch 4/50
804/804 [==============================] - 0s 540us/step - loss:

In [57]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1_1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5518 - accuracy: 0.7273
Loss: 0.5517529845237732, Accuracy: 0.7273469567298889


### 2.1.2 - 3 Layers and Activations All Relu

In [58]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  40
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 10
# hidden_nodes_layer4 = 5


nn1_2 = tf.keras.models.Sequential()

# First hidden layer
nn1_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn1_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn1_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
# nn1_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="sigmoid"))

# Output layer
nn1_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1_2.summary()


# Compile the model
nn1_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
mc = tf.keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', 
                                     save_freq='epoch',period=5)
fit_model = nn1_2.fit(X_train_scaled, y_train, callbacks=[mc], epochs=50)

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_74 (Dense)             (None, 40)                2880      
_________________________________________________________________
dense_75 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_76 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_77 (Dense)             (None, 1)                 11        
Total params: 3,921
Trainable params: 3,921
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 1s 530us/step - loss: 0.5841 - accuracy: 0.7081
Epoch 2/50
804/804 [==============================] - 0s 530us/step - loss: 0.5531 - accuracy: 0.7311
Epoch 3/50
804/804 [=========================

In [59]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5529 - accuracy: 0.7273
Loss: 0.552916407585144, Accuracy: 0.7273469567298889


### 2.1.3 - 3 Layers and Activations Mixture of Relu and Sigmoid

In [60]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  40
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 10
# hidden_nodes_layer4 = 5


nn1_3 = tf.keras.models.Sequential()

# First hidden layer
nn1_3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn1_3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn1_3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Fourth hidden layer
# nn1_3.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="sigmoid"))

# Output layer
nn1_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1_3.summary()


# Compile the model
nn1_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
mc = tf.keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', 
                                     save_freq='epoch',period=5)
fit_model = nn1_3.fit(X_train_scaled, y_train, callbacks=[mc], epochs=50)

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_78 (Dense)             (None, 40)                2880      
_________________________________________________________________
dense_79 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_80 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_81 (Dense)             (None, 1)                 11        
Total params: 3,921
Trainable params: 3,921
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 1s 584us/step - loss: 0.5771 - accuracy: 0.7176
Epoch 2/50
804/804 [==============================] - 1s 648us/step - loss: 0.5531 - accuracy: 0.7327
Epoch 3/50
804/804 [=========================

In [61]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5509 - accuracy: 0.7276
Loss: 0.5509210228919983, Accuracy: 0.727580189704895


### 2.1.4 - 4 Layers and Activations All Relu

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  40
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 10
hidden_nodes_layer4 = 5


nn1_4 = tf.keras.models.Sequential()

# First hidden layer
nn1_4.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn1_4.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn1_4.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn1_4.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn1_4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1_4.summary()


# Compile the model
nn1_4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
mc = tf.keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', 
                                     save_freq='epoch',period=5)
fit_model = nn1_4.fit(X_train_scaled, y_train, callbacks=[mc], epochs=50)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 40)                2880      
_________________________________________________________________
dense_1 (Dense)              (None, 20)                820       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 6         
Total params: 3,971
Trainable params: 3,971
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 1s 556us/step - loss: 0.5797 - accuracy: 0.7177
Epoch 2/50
804/804

In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1_4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5522 - accuracy: 0.7282
Loss: 0.5521957874298096, Accuracy: 0.7281632423400879


### 2.1.5 - 4 Layers and Activations Mixture of Relu and Sigmoid

In [64]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  40
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 10
hidden_nodes_layer4 = 5


nn1_5 = tf.keras.models.Sequential()

# First hidden layer
nn1_5.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn1_5.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn1_5.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Fourth hidden layer
nn1_5.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn1_5.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1_5.summary()


# Compile the model
nn1_5.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
mc = tf.keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', 
                                     save_freq='epoch',period=5)
fit_model = nn1_5.fit(X_train_scaled, y_train, callbacks=[mc], epochs=50)

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_87 (Dense)             (None, 40)                2880      
_________________________________________________________________
dense_88 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_89 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_90 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_91 (Dense)             (None, 1)                 6         
Total params: 3,971
Trainable params: 3,971
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 1s 549us/step - loss: 0.5969 - accuracy: 0.7023
Epoch 2/50
804/

In [65]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1_5.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5522 - accuracy: 0.7270
Loss: 0.5521947145462036, Accuracy: 0.7269970774650574


## 2.2 Optimize Model Through Bin Size and Feature Columns
*It was determined that 4 layers with all activation Relu to be the best*

### 2.2.1 Bins < 10 for Application Type and Classification 

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  40
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 10
hidden_nodes_layer4 = 5


nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 40)                2880      
_________________________________________________________________
dense_16 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_17 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_18 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 6         
Total params: 3,971
Trainable params: 3,971
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
# Train the model
mc = tf.keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', 
                                     save_freq='epoch',period=5)
fit_model = nn1.fit(X_train_scaled, y_train, callbacks=[mc], epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 538us/step - loss: 0.6062 - accuracy: 0.7077
Epoch 2/50
804/804 [==============================] - 0s 546us/step - loss: 0.5615 - accuracy: 0.7306
Epoch 3/50
804/804 [==============================] - 0s 540us/step - loss: 0.5539 - accuracy: 0.7313
Epoch 4/50
804/804 [==============================] - 0s 546us/step - loss: 0.5496 - accuracy: 0.7330
Epoch 5/50
804/804 [==============================] - 0s 566us/step - loss: 0.5479 - accuracy: 0.7339
Epoch 6/50
804/804 [==============================] - 0s 543us/step - loss: 0.5460 - accuracy: 0.7348
Epoch 7/50
804/804 [==============================] - 0s 590us/step - loss: 0.5452 - accuracy: 0.7352
Epoch 8/50
804/804 [==============================] - 0s 561us/step - loss: 0.5436 - accuracy: 0.7369
Epoch 9/50
804/804 [==============================] - 1s 652us/step - loss: 0.5438 - accuracy: 0.7357
Epoch 10/50
804/804 [==============================] - 0s 561us/step - loss: 0.542

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5526 - accuracy: 0.7289
Loss: 0.5526093244552612, Accuracy: 0.728863000869751


### 2.2.2 Bins <20 for Application Type and Classification 

In [24]:
application_df2 = pd.read_csv("../Resources/charity_data.csv")
application_df2 = application_df2.drop(['EIN','NAME'],axis=1)
typeDF = application_df2['APPLICATION_TYPE'].value_counts().to_frame().reset_index()
type10 = typeDF.loc[typeDF['APPLICATION_TYPE']<20]
application_types_to_replace = type10['index'].to_list()

# Replace in dataframe
for app in application_types_to_replace:
    application_df2['APPLICATION_TYPE'] = application_df2['APPLICATION_TYPE'].replace(app,"Other")

classDF = application_df2['CLASSIFICATION'].value_counts().to_frame().reset_index()
class10 = classDF.loc[classDF['CLASSIFICATION']<20]
classifications_to_replace = class10['index'].to_list()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df2['CLASSIFICATION'] = application_df2['CLASSIFICATION'].replace(cls,"Other")
application_df2 = pd.get_dummies(application_df2)

# Split our preprocessed data into our features and target arrays
y = application_df2['IS_SUCCESSFUL'].values
X = application_df2.drop(['IS_SUCCESSFUL'],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  40
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 10
hidden_nodes_layer4 = 5


nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
mc = tf.keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', 
                                     save_freq='epoch',period=5)
fit_model = nn2.fit(X_train_scaled, y_train, callbacks=[mc], epochs=50)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 40)                2520      
_________________________________________________________________
dense_21 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_22 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_23 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 6         
Total params: 3,611
Trainable params: 3,611
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 1s 615us/step - loss: 0.5817 - accuracy: 0.7144
Epoch 2/50
804/8

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5504 - accuracy: 0.7272
Loss: 0.550447940826416, Accuracy: 0.7272303104400635


### 2.2.3 Bins < 5 for Application Type and Classification 

In [26]:
application_df3 = pd.read_csv("../Resources/charity_data.csv")
application_df3 = application_df3.drop(['EIN','NAME'],axis=1)
typeDF = application_df3['APPLICATION_TYPE'].value_counts().to_frame().reset_index()
type10 = typeDF.loc[typeDF['APPLICATION_TYPE']<5]
application_types_to_replace = type10['index'].to_list()

# Replace in dataframe
for app in application_types_to_replace:
    application_df3['APPLICATION_TYPE'] = application_df3['APPLICATION_TYPE'].replace(app,"Other")

classDF = application_df3['CLASSIFICATION'].value_counts().to_frame().reset_index()
class10 = classDF.loc[classDF['CLASSIFICATION']<5]
classifications_to_replace = class10['index'].to_list()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df3['CLASSIFICATION'] = application_df3['CLASSIFICATION'].replace(cls,"Other")
application_df3 = pd.get_dummies(application_df3)

# Split our preprocessed data into our features and target arrays
y = application_df3['IS_SUCCESSFUL'].values
X = application_df3.drop(['IS_SUCCESSFUL'],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  40
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 10
hidden_nodes_layer4 = 5


nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
mc = tf.keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', 
                                     save_freq='epoch',period=5)
fit_model = nn3.fit(X_train_scaled, y_train, callbacks=[mc], epochs=50)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 40)                3200      
_________________________________________________________________
dense_26 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_27 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_28 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 6         
Total params: 4,291
Trainable params: 4,291
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 1s 563us/step - loss: 0.5769 - accuracy: 0.7062
Epoch 2/50
804/8

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5546 - accuracy: 0.7265
Loss: 0.5546078085899353, Accuracy: 0.7265306115150452


### 2.2.4 Drop Column  AFFILIATION

In [30]:
application_df4 = pd.read_csv("../Resources/charity_data.csv")
application_df4 = application_df4.drop(['EIN','NAME','AFFILIATION'],axis=1)
typeDF = application_df4['APPLICATION_TYPE'].value_counts().to_frame().reset_index()
type10 = typeDF.loc[typeDF['APPLICATION_TYPE']<10]
application_types_to_replace = type10['index'].to_list()

# Replace in dataframe
for app in application_types_to_replace:
    application_df4['APPLICATION_TYPE'] = application_df4['APPLICATION_TYPE'].replace(app,"Other")

classDF = application_df4['CLASSIFICATION'].value_counts().to_frame().reset_index()
class10 = classDF.loc[classDF['CLASSIFICATION']<10]
classifications_to_replace = class10['index'].to_list()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df4['CLASSIFICATION'] = application_df4['CLASSIFICATION'].replace(cls,"Other")
application_df4 = pd.get_dummies(application_df4)

# Split our preprocessed data into our features and target arrays
y = application_df4['IS_SUCCESSFUL'].values
X = application_df4.drop(['IS_SUCCESSFUL'],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  40
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 10
hidden_nodes_layer4 = 5


nn4 = tf.keras.models.Sequential()

# First hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn4.summary()

# Compile the model
nn4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
mc = tf.keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', 
                                     save_freq='epoch',period=5)
fit_model = nn4.fit(X_train_scaled, y_train, callbacks=[mc], epochs=50)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             (None, 40)                2640      
_________________________________________________________________
dense_36 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_37 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_38 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 6         
Total params: 3,731
Trainable params: 3,731
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 1s 549us/step - loss: 0.6310 - accuracy: 0.6360
Epoch 2/50
804/8

In [31]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6074 - accuracy: 0.6581
Loss: 0.6074118614196777, Accuracy: 0.6580758094787598


### 2.2.5 Drop Column Use Case

In [32]:
application_df5 = pd.read_csv("../Resources/charity_data.csv")
application_df5 = application_df5.drop(['EIN','NAME','USE_CASE'],axis=1)
typeDF = application_df5['APPLICATION_TYPE'].value_counts().to_frame().reset_index()
type10 = typeDF.loc[typeDF['APPLICATION_TYPE']<10]
application_types_to_replace = type10['index'].to_list()

# Replace in dataframe
for app in application_types_to_replace:
    application_df5['APPLICATION_TYPE'] = application_df5['APPLICATION_TYPE'].replace(app,"Other")

classDF = application_df5['CLASSIFICATION'].value_counts().to_frame().reset_index()
class10 = classDF.loc[classDF['CLASSIFICATION']<10]
classifications_to_replace = class10['index'].to_list()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df5['CLASSIFICATION'] = application_df5['CLASSIFICATION'].replace(cls,"Other")
application_df5 = pd.get_dummies(application_df5)

# Split our preprocessed data into our features and target arrays
y = application_df5['IS_SUCCESSFUL'].values
X = application_df5.drop(['IS_SUCCESSFUL'],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  40
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 10
hidden_nodes_layer4 = 5


nn5 = tf.keras.models.Sequential()

# First hidden layer
nn5.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn5.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn5.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn5.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn5.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn5.summary()

# Compile the model
nn5.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
mc = tf.keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', 
                                     save_freq='epoch',period=5)
fit_model = nn5.fit(X_train_scaled, y_train, callbacks=[mc], epochs=50)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 40)                2680      
_________________________________________________________________
dense_41 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_42 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_43 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_44 (Dense)             (None, 1)                 6         
Total params: 3,771
Trainable params: 3,771
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 1s 599us/step - loss: 0.6008 - accuracy: 0.7094
Epoch 2/50
804/8

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn5.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5577 - accuracy: 0.7275
Loss: 0.5577057600021362, Accuracy: 0.7274635434150696


### 2.2.6 Drop Column  Organization

In [34]:
application_df6 = pd.read_csv("../Resources/charity_data.csv")
application_df6 = application_df6.drop(['EIN','NAME','ORGANIZATION'],axis=1)
typeDF = application_df6['APPLICATION_TYPE'].value_counts().to_frame().reset_index()
type10 = typeDF.loc[typeDF['APPLICATION_TYPE']<10]
application_types_to_replace = type10['index'].to_list()

# Replace in dataframe
for app in application_types_to_replace:
    application_df6['APPLICATION_TYPE'] = application_df6['APPLICATION_TYPE'].replace(app,"Other")

classDF = application_df6['CLASSIFICATION'].value_counts().to_frame().reset_index()
class10 = classDF.loc[classDF['CLASSIFICATION']<10]
classifications_to_replace = class10['index'].to_list()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df6['CLASSIFICATION'] = application_df6['CLASSIFICATION'].replace(cls,"Other")
application_df6 = pd.get_dummies(application_df6)

# Split our preprocessed data into our features and target arrays
y = application_df6['IS_SUCCESSFUL'].values
X = application_df6.drop(['IS_SUCCESSFUL'],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  40
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 10
hidden_nodes_layer4 = 5


nn6 = tf.keras.models.Sequential()

# First hidden layer
nn6.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn6.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn6.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn6.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn6.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn6.summary()

# Compile the model
nn6.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
mc = tf.keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', 
                                     save_freq='epoch',period=5)
fit_model = nn6.fit(X_train_scaled, y_train, callbacks=[mc], epochs=50)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (None, 40)                2720      
_________________________________________________________________
dense_46 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_47 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_48 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_49 (Dense)             (None, 1)                 6         
Total params: 3,811
Trainable params: 3,811
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 1s 575us/step - loss: 0.5844 - accuracy: 0.7151
Epoch 2/50
804/8

In [35]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn6.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5607 - accuracy: 0.7229
Loss: 0.5606698393821716, Accuracy: 0.7229154706001282


### 2.2.7 Drop Column Status

In [36]:
application_df7 = pd.read_csv("../Resources/charity_data.csv")
application_df7 = application_df7.drop(['EIN','NAME','STATUS'],axis=1)
typeDF = application_df7['APPLICATION_TYPE'].value_counts().to_frame().reset_index()
type10 = typeDF.loc[typeDF['APPLICATION_TYPE']<10]
application_types_to_replace = type10['index'].to_list()

# Replace in dataframe
for app in application_types_to_replace:
    application_df7['APPLICATION_TYPE'] = application_df7['APPLICATION_TYPE'].replace(app,"Other")

classDF = application_df7['CLASSIFICATION'].value_counts().to_frame().reset_index()
class10 = classDF.loc[classDF['CLASSIFICATION']<10]
classifications_to_replace = class10['index'].to_list()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df7['CLASSIFICATION'] = application_df7['CLASSIFICATION'].replace(cls,"Other")
application_df7 = pd.get_dummies(application_df7)

# Split our preprocessed data into our features and target arrays
y = application_df7['IS_SUCCESSFUL'].values
X = application_df7.drop(['IS_SUCCESSFUL'],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  40
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 10
hidden_nodes_layer4 = 5


nn7 = tf.keras.models.Sequential()

# First hidden layer
nn7.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn7.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn7.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn7.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn7.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn7.summary()

# Compile the model
nn7.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
mc = tf.keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', 
                                     save_freq='epoch',period=5)
fit_model = nn7.fit(X_train_scaled, y_train, callbacks=[mc], epochs=50)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_50 (Dense)             (None, 40)                2840      
_________________________________________________________________
dense_51 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_52 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_53 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_54 (Dense)             (None, 1)                 6         
Total params: 3,931
Trainable params: 3,931
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 1s 560us/step - loss: 0.5874 - accuracy: 0.7132
Epoch 2/50
804/

In [37]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn7.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5521 - accuracy: 0.7270
Loss: 0.5521289706230164, Accuracy: 0.7269970774650574


### 2.2.8 Drop Column Income Amount

In [38]:
application_df8 = pd.read_csv("../Resources/charity_data.csv")
application_df8 = application_df8.drop(['EIN','NAME','INCOME_AMT'],axis=1)
typeDF = application_df8['APPLICATION_TYPE'].value_counts().to_frame().reset_index()
type10 = typeDF.loc[typeDF['APPLICATION_TYPE']<10]
application_types_to_replace = type10['index'].to_list()

# Replace in dataframe
for app in application_types_to_replace:
    application_df8['APPLICATION_TYPE'] = application_df8['APPLICATION_TYPE'].replace(app,"Other")

classDF = application_df8['CLASSIFICATION'].value_counts().to_frame().reset_index()
class10 = classDF.loc[classDF['CLASSIFICATION']<10]
classifications_to_replace = class10['index'].to_list()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df8['CLASSIFICATION'] = application_df8['CLASSIFICATION'].replace(cls,"Other")
application_df8 = pd.get_dummies(application_df8)

# Split our preprocessed data into our features and target arrays
y = application_df8['IS_SUCCESSFUL'].values
X = application_df8.drop(['IS_SUCCESSFUL'],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  40
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 10
hidden_nodes_layer4 = 5


nn8 = tf.keras.models.Sequential()

# First hidden layer
nn8.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn8.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn8.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn8.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn8.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn8.summary()

# Compile the model
nn8.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
mc = tf.keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', 
                                     save_freq='epoch',period=5)
fit_model = nn8.fit(X_train_scaled, y_train, callbacks=[mc], epochs=50)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_55 (Dense)             (None, 40)                2520      
_________________________________________________________________
dense_56 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_57 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_58 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_59 (Dense)             (None, 1)                 6         
Total params: 3,611
Trainable params: 3,611
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 1s 563us/step - loss: 0.5818 - accuracy: 0.7133
Epoch 2/50
804/

In [39]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn8.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5536 - accuracy: 0.7280
Loss: 0.5536155104637146, Accuracy: 0.7280466556549072


### 2.2.9 Drop Column Special Considerations

In [40]:
application_df9 = pd.read_csv("../Resources/charity_data.csv")
application_df9 = application_df9.drop(['EIN','NAME','SPECIAL_CONSIDERATIONS'],axis=1)
typeDF = application_df9['APPLICATION_TYPE'].value_counts().to_frame().reset_index()
type10 = typeDF.loc[typeDF['APPLICATION_TYPE']<10]
application_types_to_replace = type10['index'].to_list()

# Replace in dataframe
for app in application_types_to_replace:
    application_df9['APPLICATION_TYPE'] = application_df9['APPLICATION_TYPE'].replace(app,"Other")

classDF = application_df9['CLASSIFICATION'].value_counts().to_frame().reset_index()
class10 = classDF.loc[classDF['CLASSIFICATION']<10]
classifications_to_replace = class10['index'].to_list()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df9['CLASSIFICATION'] = application_df9['CLASSIFICATION'].replace(cls,"Other")
application_df9 = pd.get_dummies(application_df9)

# Split our preprocessed data into our features and target arrays
y = application_df9['IS_SUCCESSFUL'].values
X = application_df9.drop(['IS_SUCCESSFUL'],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  40
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 10
hidden_nodes_layer4 = 5


nn9 = tf.keras.models.Sequential()

# First hidden layer
nn9.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn9.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn9.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn9.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn9.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn9.summary()

# Compile the model
nn9.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
mc = tf.keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', 
                                     save_freq='epoch',period=5)
fit_model = nn9.fit(X_train_scaled, y_train, callbacks=[mc], epochs=50)

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_60 (Dense)             (None, 40)                2800      
_________________________________________________________________
dense_61 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_62 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_63 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_64 (Dense)             (None, 1)                 6         
Total params: 3,891
Trainable params: 3,891
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 1s 558us/step - loss: 0.5862 - accuracy: 0.7054
Epoch 2/50
804/

In [41]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn9.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5543 - accuracy: 0.7294
Loss: 0.5542712211608887, Accuracy: 0.7294460535049438


### 2.2.10 Drop Column Ask Amount

In [42]:
application_df10 = pd.read_csv("../Resources/charity_data.csv")
application_df10 = application_df10.drop(['EIN','NAME','ASK_AMT'],axis=1)
typeDF = application_df10['APPLICATION_TYPE'].value_counts().to_frame().reset_index()
type10 = typeDF.loc[typeDF['APPLICATION_TYPE']<10]
application_types_to_replace = type10['index'].to_list()

# Replace in dataframe
for app in application_types_to_replace:
    application_df10['APPLICATION_TYPE'] = application_df10['APPLICATION_TYPE'].replace(app,"Other")

classDF = application_df10['CLASSIFICATION'].value_counts().to_frame().reset_index()
class10 = classDF.loc[classDF['CLASSIFICATION']<10]
classifications_to_replace = class10['index'].to_list()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df10['CLASSIFICATION'] = application_df10['CLASSIFICATION'].replace(cls,"Other")
application_df10 = pd.get_dummies(application_df10)

# Split our preprocessed data into our features and target arrays
y = application_df10['IS_SUCCESSFUL'].values
X = application_df10.drop(['IS_SUCCESSFUL'],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  40
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 10
hidden_nodes_layer4 = 5


nn10 = tf.keras.models.Sequential()

# First hidden layer
nn10.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn10.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn10.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn10.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn10.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn10.summary()

# Compile the model
nn10.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
mc = tf.keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', 
                                     save_freq='epoch',period=5)
fit_model = nn10.fit(X_train_scaled, y_train, callbacks=[mc], epochs=50)

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_65 (Dense)             (None, 40)                2840      
_________________________________________________________________
dense_66 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_67 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_68 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_69 (Dense)             (None, 1)                 6         
Total params: 3,931
Trainable params: 3,931
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 1s 581us/step - loss: 0.5842 - accuracy: 0.7090
Epoch 2/50
804/

In [43]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn10.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5518 - accuracy: 0.7291
Loss: 0.5517695546150208, Accuracy: 0.7290962338447571


# 3.0 Save Model

In [44]:
# Export our model to HDF5 file
nn9.save("../Resources/AlphabetSoupCharity_Optimization.h5")